In [1]:
import pickle as pkl
import pk_plot_library as pplib
import numpy as np

pplib.imports()

DRAFTPATH = '/Users/cosinga/code/hicolor_draft_two/figures/'
SAVEPATH = '/Users/cosinga/code/hcolor/fig_md_analysis/binth/'

In [2]:
ds = pkl.load(open('/Users/cosinga/code/hcolor/fig_md_analysis/11-1_pkdatasort.pkl', 'rb'))
ds.getAttrVals('path')

['fiducial',
 'allgals',
 'HIbt',
 'species',
 'galbt',
 'MAS',
 'low-2',
 'colordef',
 'axistest1',
 'convtest',
 'low-3',
 'midgr',
 'axistest2']